In [13]:
import pandas as pd
import numpy as np
import re
import nltk

import datasets
import evaluate
import accelerate

import Bartscore as bartscore ## code for Bartscore


e:\Anaconda\envs\CudaSupEnv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


OSError: [WinError 1455] Het wisselbestand is te klein voor het voltooien van deze bewerking. Error loading "e:\Anaconda\envs\CudaSupEnv\lib\site-packages\torch\lib\cudnn_cnn_infer64_8.dll" or one of its dependencies.

In [6]:
data = pd.DataFrame()
data["RDF"] = pd.read_csv('E:\ArriaThesis\MscThesis\Manual_Eval\T5-dutch\Output Examples\yhavinga_t5-v1.1-base-dutch-cased_rdfs.txt', sep='delimiter', header=None, encoding='UTF-8')
data["Predictions"] = pd.read_csv('E:\ArriaThesis\MscThesis\Manual_Eval\T5-dutch\Output Examples\yhavinga_t5-v1.1-base-dutch-cased_decode_predictions.txt', sep='delimiter', header=None, encoding='UTF-8')
data['Real'] = pd.read_csv('E:\ArriaThesis\MscThesis\Manual_Eval\T5-dutch\Output Examples\yhavinga_t5-v1.1-base-dutch-cased_true_articles.txt', sep='delimiter', header=None, encoding='UTF-8')

In [7]:
def load_eval_metrics():
    """
    Loads in all metrics that will be used later on during evaluation. This is seperated to not load in the metrics a dozen of times during training.
    """
    bleu = datasets.load_metric("bleu")
    rouge = evaluate.load('rouge')
    meteor = evaluate.load('meteor')
    perplexity = evaluate.load("perplexity", module_type="metric")
    bertscore = evaluate.load("bertscore")
    bart_scorer = bartscore.BARTScorer(checkpoint='facebook/bart-base') 

    print('LOGGING: load_eval_metrics DONE \n')

    return bleu, rouge, meteor, perplexity, bertscore, bart_scorer

In [8]:
def postprocess_text(preds, labels):
    """
    Supplementary Method called in decode_text.

    Returns list of split decoded labels and predictions for evaluation
    """
    preds = [pred.split() for pred in preds]
    labels = [[label.split()] for label in labels]
    return preds, labels

In [9]:
def evaluate_texts(decoded_preds, decoded_labels):
    """
    Calculates metrics given a list of decoded predictions and decoded labels
    """
    #post_process for BLEU
    blue_preds, blue_labels = postprocess_text(decoded_preds,  decoded_labels)

    # setup metrics for use
    #bleu, rouge, meteor,perplexity, bertscore, bart_scorer = load_eval_metrics()

    #Calculate the metrics
    print(f'\n LOGGING: Calculating Blue')
    bleu_output = bleu.compute(predictions=blue_preds, references=blue_labels)
    print(f'\n LOGGING: Calculating Rouge')
    rouge_output = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    print(f'\n LOGGING: Calculating Meteor')
    meteor_output = meteor.compute(predictions=decoded_preds, references=decoded_labels)
    print(f'\n LOGGING: Calculating Perplexity')
    perp_output = perplexity.compute(predictions=decoded_preds, model_id='gpt2')
    print(f'\n LOGGING: Calculating Bertscore')
    bertscore_output = bertscore.compute(predictions=decoded_preds, references=decoded_labels, lang="en")
    print(f'\n LOGGING: Calculating Bartscore')
    bart_scores_output = bart_scorer.score(srcs=decoded_preds, tgts=decoded_labels, batch_size=16)

    print(f'\n LOGGING: Done calculations')

    return bleu_output, rouge_output, meteor_output, perp_output, bertscore_output, bart_scores_output

In [11]:
def Set_evaluation():
    #dataframe = prepare_dataframe()
    dataframe = data.copy()
    # setup metrics for use
    bleu, rouge, meteor,perplexity, bertscore, bart_scorer = load_eval_metrics()

    for row_index, row in enumerate(dataframe):
        ## Calculate each metric
        bleu_output, rouge_output, meteor_output, perp_output, bertscore_output, bart_scores_output = evaluate_texts(row["Predictions"], row["Real"])
        ## Add the metric to the correct column
        dataframe['BLEU'][row_index] = bleu_output
        dataframe['Rouge'][row_index] = rouge_output
        dataframe['METEOR'][row_index] = meteor_output  
        dataframe['Perplexity'][row_index] = perp_output
        dataframe['BertScore'][row_index] = bertscore_output  
        dataframe['BartScore'][row_index] = bart_scores_output          

In [12]:
Set_evaluation()

NameError: name 'datasets' is not defined